In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt

# Carrega os resultados
with open("resultados.json", "r", encoding="utf-8") as f:
    resultados = json.load(f)

Gráfico 1: Tempo médio por combinação

In [ ]:
labels = []
tempos_medios = []

for r in resultados:
    label = f"{r['ocr']} + {r['grade']}"
    labels.append(label)
    tempos_medios.append(np.mean(r["tempos"]))

plt.figure(figsize=(8, 4))
plt.bar(labels, tempos_medios, color="skyblue")
plt.xticks(rotation=45, ha="right")
plt.title("Tempo médio por combinação de métodos")
plt.ylabel("Tempo médio (s)")
plt.tight_layout()
plt.show()

Gráfico 2: Pontuação por partida (de um método específico)

In [ ]:
# Pegando o primeiro método como exemplo
r = resultados[0]
plt.figure(figsize=(8, 4))
plt.bar(range(1, len(r["pontuacoes"]) + 1), r["pontuacoes"], color="orange")
plt.title(f"Pontuação por partida ({r['ocr']} + {r['grade']})")
plt.xlabel("Partida")
plt.ylabel("Pontuação")
plt.tight_layout()
plt.show()

 Gráfico 3: Maior número por partida


In [ ]:
plt.figure(figsize=(8, 4))
plt.bar(range(1, len(r["maiores_numeros"]) + 1), r["maiores_numeros"], color="green")
plt.title(f"Maior número por partida ({r['ocr']} + {r['grade']})")
plt.xlabel("Partida")
plt.ylabel("Maior número")
plt.tight_layout()
plt.show()

Gráfico 4: Total de falhas por combinação

In [ ]:
labels = []
falhas = []

for r in resultados:
    label = f"{r['ocr']} + {r['grade']}"
    labels.append(label)
    falhas.append(r["falhas_grid"])

plt.figure(figsize=(8, 4))
plt.bar(labels, falhas, color="red")
plt.xticks(rotation=45, ha="right")
plt.title("Falhas de grid por combinação de métodos")
plt.ylabel("Total de falhas")
plt.tight_layout()
plt.show()
